Old and out of date -- do not use!

Initial development notebook before everything was scriptified

In [ ]:
from __future__ import print_function

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import matplotlib

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1337)

import keras
from keras.models import Model
from keras.layers import Dense, Input, Reshape, Lambda
from keras.layers.convolutional import Conv2D
from keras import backend as K

import medleydb as mdb
import os
import pandas


RANDOM_STATE = 42

In [ ]:
import glob
import pescador
from medleydb import utils

def keras_generator(data_list, input_patch_size,
                    output_patch_size,
                    with_replacement=True,
                    batch_size=32):

    streams = []
    for fpath_in, fpath_out in data_list:
        streams.append(
            pescador.Streamer(
                patch_generator, fpath_in, fpath_out,
                input_patch_size=input_patch_size,
                output_patch_size=output_patch_size,
                batch_size=batch_size
            )
        )

    stream_mux = pescador.Mux(
        streams, 10, with_replacement=with_replacement, lam=500,
        random_state=RANDOM_STATE
    )

#     buffered_streamer = pescador.BufferedStreamer(stream_mux, batch_size)

    for batch in stream_mux.tuples('X', 'Y'):
        yield batch


def __grab_patch_output(f, t, n_f, n_t, y_data):
    return y_data[f: f + n_f, t: t + n_t][np.newaxis, :, :]


def __grab_patch_input(f, t, n_f, n_t, n_harms, x_data):
    return np.transpose(
        x_data[:, f: f + n_f, t: t + n_t], (1, 2, 0)
    )[np.newaxis, :, :, :]


def patch_generator(fpath_in, fpath_out, input_patch_size, output_patch_size, batch_size):
    data_in = np.load(fpath_in)#, mmap_mode='r')
    data_out = np.load(fpath_out)#, mmap_mode='r')
    
    n_harms, n_freqs, n_times = data_in.shape
    n_f_in, n_t_in = input_patch_size
    n_f_out, n_t_out = output_patch_size

    f_shift = n_f_in - n_f_out
    t_shift = n_t_in - n_t_out

#     while True:
    t_vals = np.arange(0, n_times - n_t_in)
    np.random.shuffle(t_vals)
    for t in t_vals:
#         f = np.random.randint(0, n_freqs - n_f_in)
        f = 0
#         t = np.random.randint(0, n_times - n_t_in)

        x = __grab_patch_input(
            f, t, n_f_in, n_t_in, n_harms, data_in
        )
        y = __grab_patch_output(
            f + f_shift, t + t_shift, n_f_out, n_t_out, data_out
        )
        yield dict(X=x, Y=y)


def get_file_paths(mtrack_list, data_path):
    file_paths = []
    for track_id in mtrack_list:
        input_path = glob.glob(
            os.path.join(data_path, 'inputs', "{}*_input.npy".format(track_id))
        )
        output_path = glob.glob(
            os.path.join(data_path, 'outputs', "{}*_output.npy".format(track_id))
        )
        
        if len(input_path) == 1 and len(output_path) == 1:
            input_path = input_path[0]
            output_path = output_path[0]
            file_paths.append((input_path, output_path))
            
    return file_paths

In [ ]:
class Data(object):

    def __init__(self, mtrack_list, data_path, input_patch_size,
                 output_patch_size, batch_size):

        self.mtrack_list = mtrack_list
        self.input_patch_size = input_patch_size
        self.output_patch_size = output_patch_size
        self.batch_size = batch_size

        self.data_path = data_path
        
        (self.train_set,
         self.validation_set,
         self.test_set) = self._train_val_test_split()

        self.train_files = get_file_paths(self.train_set, self.data_path)
        self.validation_files = get_file_paths(
            self.validation_set, self.data_path
        )
        self.test_files = get_file_paths(self.test_set, self.data_path)

    def _train_val_test_split(self):
        full_list = []
        print(len(self.mtrack_list))
        for m in self.mtrack_list:
            globbed = get_file_paths([m], self.data_path)
            if len(globbed) > 0:
                full_list.append(m)

        self.full_list = full_list
        print(len(full_list))
        mtracks = list(mdb.load_multitracks(full_list))
        test_potentials = [
            m.track_id for m in mtracks if m.dataset_version == 'V1'
        ]
        all_others = [
            m.track_id for m in mtracks if m.dataset_version != 'V1'
        ]

        split1 = utils.artist_conditional_split(
            trackid_list=test_potentials, test_size=0.2,
            num_splits=1, random_state=RANDOM_STATE
        )

        test_set = split1[0]['test']
        remaining_tracks = split1[0]['train'] + all_others

        split2 = utils.artist_conditional_split(
            trackid_list=remaining_tracks, test_size=0.15,
            num_splits=1, random_state=RANDOM_STATE
        )

        train_set = split2[0]['train']
        validation_set = split2[0]['test']

        return train_set, validation_set, test_set

    def get_train_generator(self):
        return keras_generator(
            self.train_files,
            input_patch_size=self.input_patch_size,
            output_patch_size=self.output_patch_size,
            batch_size=self.batch_size
        )

    def get_validation_generator(self):
        return keras_generator(
            self.validation_files,
            input_patch_size=self.input_patch_size,
            output_patch_size=self.output_patch_size,
            batch_size=self.batch_size
        )

    def get_test_generator(self):
        return keras_generator(
            self.test_files,
            input_patch_size=self.input_patch_size,
            output_patch_size=self.output_patch_size,
            batch_size=self.batch_size
        )

# load data

In [ ]:
data_path = "/scratch/rmb456/multif0_ismir2017/training_data_with_blur/multif0_complete/"
mtrack_list = mdb.TRACK_LIST_V1 + mdb.TRACK_LIST_V2 + mdb.TRACK_LIST_EXTRA #+ mdb.TRACK_LIST_BACH10
dat = Data(
    mtrack_list, data_path, input_patch_size=(360, 50),
    output_patch_size=(360, 50), batch_size=10
)

In [ ]:
# print(len(dat.train_set))
# print(len(dat.validation_set))
print(dat.test_set)

In [ ]:
train_generator = dat.get_train_generator()
validation_generator = dat.get_validation_generator()
test_generator = dat.get_test_generator()

# Build Model

In [ ]:
from keras.layers.advanced_activations import LeakyReLU
# LeakyReLU(alpha=0.3)

In [ ]:
input_shape = (None, None, 6)
inputs = Input(shape=input_shape)

y1 = Conv2D(
    64, (5, 5), padding='same', activation='relu', name='bendy1'
)(inputs)
y2 = Conv2D(
    64, (5, 5), padding='same', activation='relu', name='bendy2'
)(y1)
# y3 = Conv2D(
#     8, (70, 3), padding='same', activation='relu', name='distribute', kernel_initializer='ones'
# )(y2)
y4 = Conv2D(
    64, (3, 3), padding='same', activation='relu', name='smoothy1'
)(y2)
y5 = Conv2D(
    64, (3, 3), padding='same', activation='relu', name='smoothy2'
)(y4)
y6 = Conv2D(
    1, (1, 1), padding='same', activation='sigmoid', name='squishy'
)(y5)
predictions = Lambda(lambda x: K.squeeze(x, axis=3))(y6)

model = Model(inputs=inputs, outputs=predictions)

# Compile Model

In [ ]:
def bkld(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), 1.0 - K.epsilon())
    y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
    return K.mean(K.mean(-1.0*y_true* K.log(y_pred) - (1.0 - y_true) * K.log(1.0 - y_pred), axis=-1), axis=-1)

def soft_binary_accuracy(y_true, y_pred):
    return K.mean(K.mean(K.equal(K.round(y_true), K.round(y_pred)),  
        axis=-1), axis=-1)

def smart_sum(x, axis=None, keepdims=False):
    axis = K.tensorflow_backend._normalize_axis(axis, K.ndim(x))
    if x.dtype.base_dtype == K.tf.bool:
        x = K.tf.cast(x, K.floatx())
    return K.tf.reduce_sum(x, reduction_indices=axis, keep_dims=keepdims)


def mirex_acc_loss(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    n_est = K.sum(K.round(K.clip(y_pred, 0, 1)))
    n_ref = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (n_est + n_ref - true_positives + K.epsilon())

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def fbeta_score(y_true, y_pred, beta=0.5):
    """Computes the F score.

    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.

    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.

    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

In [ ]:
# loss binary cross entropy
model.compile(loss=bkld,
              metrics=['mse', soft_binary_accuracy],#, fbeta_score, mirex_acc_loss],
              optimizer='adam')

In [ ]:
model.summary(line_length=80)

# Fit Model

In [ ]:
model_path = "/home/rmb456/repos/multif0/experiment_output/notebook_test4"
if not os.path.exists(model_path):
    os.mkdir(model_path)
samples_per_epoch = 512  # 512 batches, batches size 32
nb_epochs = 100 # up to 100 epochs
nb_val_samples = 512 # 1000

history = model.fit_generator(
    train_generator, samples_per_epoch, epochs=nb_epochs, verbose=1,
    validation_data=validation_generator, validation_steps=nb_val_samples,
    callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_path, 'test_model.pkl'),
                                           save_best_only=True, verbose=1),
               keras.callbacks.ReduceLROnPlateau(patience=5, verbose=1),
               keras.callbacks.EarlyStopping(patience=15, verbose=0)]
)

In [ ]:
cd ../deepsalience/

In [ ]:
import core as C
reload(C)

In [ ]:
model.load_weights(os.path.join(model_path, "test_model.pkl"))
C.plot_metrics_epochs(history, os.path.join(model_path, "multif0_exper1_loss.pdf"))

In [ ]:
C.get_model_metrics(dat, model, os.path.join(model_path, "multif0_exper1_model_scores.csv"))

In [ ]:
C.get_all_multif0_metrics(
    dat.test_files, model,
    model_path,
    os.path.join(model_path, "multif0_exper1_scores.csv"),
    os.path.join(model_path, "multif0_exper1_score_summary.csv"))

In [ ]:
bach10_files = get_file_paths(mdb.TRACK_LIST_BACH10, dat.data_path)
C.get_all_multif0_metrics(
    bach10_files, model,
    model_path,
    os.path.join(model_path, "bach10_scores.csv"),
    os.path.join(model_path, "bach10_score_summary.csv"), create_pred=True)

In [ ]:
reload(matplotlib)
# plot filters
conv_layer = model.get_layer(name='smoothy2')
weights = conv_layer.get_weights()
weight_array = weights[0]
plt.figure(figsize=(15, 15))
for i in range(64):
    plt.subplot(8, 8, i+1)
    plt.imshow(weight_array[:, :, 0, i], origin='lower')
    plt.axis('square')

plt.savefig('/home/rmb456/repos/multif0/filters4.pdf', format='pdf')

In [ ]:
conv_layer = model.get_layer(name='distribute')
weights = conv_layer.get_weights()
weight_array = weights[0]
plt.figure(figsize=(15, 15))
for i in range(8):
    plt.subplot(4, 2, i+1)
    plt.imshow(weight_array[:, :, 0, i], origin='lower')
    plt.axis('auto')

plt.savefig('/home/rmb456/repos/multif0/filters2.pdf', format='pdf')

In [ ]:
# plot filters
conv_layer = model.get_layer(name='smoothy1')
weights = conv_layer.get_weights()
weight_array = weights[0]
plt.figure(figsize=(15, 15))
for i in range(64):
    plt.subplot(8, 8, i+1)
    plt.imshow(weight_array[:, :, 0, i], origin='lower')
    plt.axis('square')

plt.show()

In [ ]:
def plot_stuff(X, Y, Y_pred):
    sns.set_style('white')
    plt.figure(figsize=(15, 15))
    
    plt.subplot(3, 1, 1)
    plt.title("prediction")
    plt.imshow(Y_pred[0], origin='lower', cmap='hot', vmin=0, vmax=1)
    plt.axis('auto')
#     plt.xlim(a, b)
    plt.colorbar()

    plt.subplot(3, 1, 2)
    plt.title('target')
    plt.imshow(Y[0], origin='lower', cmap='hot', vmin=0, vmax=1)
    plt.axis('auto')
#     plt.xlim(a, b)
    plt.colorbar()

    plt.subplot(3, 1, 3)
    plt.title('input')
    plt.imshow(X[0, :, :, 0], origin='lower', cmap='hot', vmin=0, vmax=1)
    plt.axis('auto')
#     plt.xlim(a, b)
    plt.colorbar()
    plt.show()

In [ ]:
for X, Y in validation_generator:
    Y_pred = model.predict(X)
    plot_stuff(X, Y, Y_pred)
    break
    if np.sum(Y[0].flatten()) > 0:
        plot_stuff(X, Y, Y_pred)
        break

In [ ]:
for X, Y in train_generator:
    for x, y in zip(X, Y):
        if np.sum(y.flatten()) > 0:
            plt.figure(figsize=(15, 15))
            plt.subplot(2, 1, 1)
            plt.imshow(x[:, :, 0], origin='lower', cmap='hot')
            plt.axis('square')

            plt.subplot(2, 1, 2)
            plt.imshow(y, origin='lower', cmap='hot')
            plt.axis('square')
            break

# Evaluate Model

In [ ]:
model.count_params()

In [ ]:
def get_full_track_prediction(fpath_in, fpath_out, model):
    
    data_in = np.load(fpath_in, mmap_mode='r')
    n_harms, n_freqs, n_times = data_in.shape
    n_f, n_t = (20, 20)

    prediction = np.zeros((n_freqs, n_times))

    cqt_patch_generator = stride_tf(fpath_in, fpath_out, (26, 26), (20, 20))

    for d in cqt_patch_generator:
        f = d['f']
        t = d['t']
        print("{}, {}".format(f, t))
        y_pred = model.predict(d['X']).reshape(n_f, n_t)
        prediction[f: f + n_f, t: t + n_t] = y_pred

    return prediction
    

In [ ]:
fpath_in, fpath_out = dat.test_files[0]
test_prediction = get_full_track_prediction(fpath_in, fpath_out, model)

In [ ]:
fpath_in

In [ ]:
a, b = (0, 25000)
ground_truth = np.load(fpath_out, mmap_mode='r')
input_data = np.load(fpath_in, mmap_mode='r')

plt.figure(figsize=(15, 15))
plt.subplot(3, 1, 1)
plt.imshow(1.0 - test_prediction[:, a:b], origin='lower', cmap='hot')
# plt.imshow((1.0 - test_prediction)**1000, origin='lower', cmap='hot')
plt.axis('auto')
plt.xlim(a, b)
# plt.colorbar()

plt.subplot(3, 1, 2)
plt.imshow(ground_truth[:, a:b], origin='lower', cmap='hot')
plt.axis('auto')
plt.xlim(a, b)

plt.subplot(3, 1, 3)
plt.imshow(input_data[0, :, a:b], origin='lower', cmap='hot')
plt.axis('auto')
plt.xlim(a, b)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.imshow(dat3['data_in'][0], origin='lower', cmap='hot')
plt.axis('auto')
plt.xlim(0, 10000)

In [ ]:
plt.imshow(dat3['data_out'], origin='lower', cmap='hot')
plt.axis('auto')
plt.xlim(0, 10000)

In [ ]:
import numpy as np

In [ ]:
test = np.zeros((100, 100))

In [ ]:
for i in range(50):
    test[i, i] = 1
for j in range(50, 100):
    test[50, j] = 1

In [ ]:
test

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(test, origin='lower', interpolation='none')
plt.axis('auto')

In [ ]:
from scipy.ndimage import filters

In [ ]:
test_filt = filters.gaussian_filter1d(test, 2, axis=0, mode='constant')
test_filt = test_filt/np.max(test_filt)

In [ ]:
plt.figure(figsize=(15, 8))
plt.imshow(test_filt, origin='lower', interpolation='none')
plt.axis('auto')
plt.colorbar()

In [ ]:
rm /scratch/rmb456/multif0_ismir2017/training_data_with_blur/multif0_complete/CelestialShore_DieForUs_multif0_complete.npz